Code 

3D wheel

In [8]:
# Parameters
vf =  0.3
γ_evo = 0.1 
max_steps = 10
α_coeff = γ_evo*max_steps
iter_mod = 50
D = 3
mesh_name = "Wheel_3d.msh"
mesh_file = (@__DIR__)*"/Models/$mesh_name"

"c:\\Users\\IIT BBSR\\Desktop\\Amiya\\BTP\\Julia\\Level_Set_Method\\Cut_FEM\\3D_Wheel/Models/Wheel_3d.msh"

In [9]:
# Output Path
path = "../../../../../Result/3D_Wheel"

#  Directory to store visualization file
files_path = path*"_data/"
isdir(files_path) || mkpath(files_path)

#  Directory to store optimized model
model_path = path*"_model/"
isdir(model_path) || mkpath(model_path)

"../../../../../Result/3D_Wheel_model"

In [10]:
using GridapGmsh.GridapGmsh: GmshDiscreteModel
using Gridap.Geometry: UnstructuredDiscreteModel
using GridapTopOpt: update_labels!
using Gridap.Visualization: writevtk

In [11]:
# Load mesh
model = GmshDiscreteModel(mesh_file)
model = UnstructuredDiscreteModel(model)
f_diri(x) =
((cos(30π/180)<=x[1]<=cos(15π/180)) && abs(x[2] - sqrt(1-x[1]^2))<1e-4) ||
((cos(97.5π/180)<=x[1]<=cos(82.5π/180)) && abs(x[2] - sqrt(1-x[1]^2))<1e-4) ||
((cos(165π/180)<=x[1]<=cos(150π/180)) && abs(x[2] - sqrt(1-x[1]^2))<1e-4) ||
((cos(142.5π/180)<=x[1]<=cos(127.5π/180)) && abs(x[2] - -sqrt(1-x[1]^2))<1e-4) ||
((cos(52.5π/180)<=x[1]<=cos(37.5π/180)) && abs(x[2] - -sqrt(1-x[1]^2))<1e-4)
update_labels!(1,model,f_diri,"Gamma_D_new")
writevtk(model,model_path*"model")

Info    : Reading 'c:\Users\IIT BBSR\Desktop\Amiya\BTP\Julia\Level_Set_Method\Cut_FEM\3D_Wheel/Models/Wheel_3d.msh'...
Info    : 15 entities
Info    : 219664 nodes
Info    : 1328560 elements                                                                                    
Info    : Done reading 'c:\Users\IIT BBSR\Desktop\Amiya\BTP\Julia\Level_Set_Method\Cut_FEM\3D_Wheel/Models/Wheel_3d.msh'


4-element Vector{Vector{String}}:
 ["../../../../../Result/3D_Wheel_model/model_0.vtu"]
 ["../../../../../Result/3D_Wheel_model/model_1.vtu"]
 ["../../../../../Result/3D_Wheel_model/model_2.vtu"]
 ["../../../../../Result/3D_Wheel_model/model_3.vtu"]

In [14]:
using Gridap.Geometry: Triangulation
using GridapTopOpt: get_element_diameters
using GridapTopOpt: get_element_diameter_field

In [ ]:
# Get triangulation and element size
Ω_bg = Triangulation(model)
hₕ = get_element_diameter_field(model)
hmin = minimum(get_element_diameters(model));

0.014743079216138905

In [19]:
using Gridap.FESpaces
using Gridap.ReferenceFEs

In [33]:
# Space
V_φ = FESpace(model, ReferenceFE(lagrangian, Float64, 1))
V_reg = TestFESpace(model,reffe_scalar;dirichlet_tags=["Gamma_N",])
U_reg = TrialFESpace(V_reg);

In [34]:
using Gridap.FESpaces: interpolate

In [36]:
# Cut the background model_f1((x,y,z),q,r) = - cos(q*π*x)*cos(q*π*y)*cos(q*π*z)/q - r/q
f1((x,y,z),q,r) = - cos(q*π*x)*cos(q*π*y)*cos(q*π*z)/q - r/q
f2((x,y,z)) = -sqrt(x^2+y^2)+0.9
φh = interpolate(x->min(f1(x,4,0.1),f2(x)),V_φ);

In [38]:
using GridapTopOpt

In [39]:
# Check LS
GridapTopOpt.correct_ls!(φh)

In [ ]:
using Gridap
using GridapEmbedded.Interfaces: PHYSICAL
using GridapEmbedded.Interfaces: EmbeddedBoundary
using GridapEmbedded.Interfaces: GhostSkeleton
using GridapEmbedded.Interfaces: ACTIVE

In [50]:
# Setup integration meshes and measures
order = 1
degree = 2*(order+1)
Γ_N = BoundaryTriangulation(model,tags=["Gamma_N",])
dΓ_N = Measure(Γ_N,degree)
dΩ_bg = Measure(Ω_bg,degree)
Ω_data = EmbeddedCollection(model,φh) do cutgeo,cutgeo_facets,_φh
Ω = DifferentiableTriangulation(Triangulation(cutgeo,PHYSICAL),V_φ)
Γ  = DifferentiableTriangulation(EmbeddedBoundary(cutgeo),V_φ)
Γg = GhostSkeleton(cutgeo)
Ω_act = Triangulation(cutgeo,ACTIVE)
# Isolated volumes
φ_cell_values = map(get_cell_dof_values,local_views(_φh))
ψ,_ = get_isolated_volumes_mask_polytopal(model,φ_cell_values,["Gamma_D_new",])
(;
    :Ω_act => Ω_act,
    :Ω     => Ω,
    :dΩ    => Measure(Ω,degree),
    :Γg    => Γg,
    :dΓg   => Measure(Γg,degree),
    :n_Γg  => get_normal_vector(Γg),
    :Γ     => Γ,
    :dΓ    => Measure(Γ,degree),
    :n_Γ   => get_normal_vector(Γ),
    :ψ     => ψ
)
end

UndefVarError: UndefVarError: `local_views` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing GridapDistributed in the current active module Main

In [ ]:
# Setup spaces
reffe_d = ReferenceFE(lagrangian,VectorValue{D,Float64},order)
function build_spaces(Ω_act)
    V = TestFESpace(Ω_act,reffe_d,conformity=:H1,dirichlet_tags=["Gamma_D_new",])
    U = TrialFESpace(V)
    return U,V
end